In [ ]:
import networkx as nx
import numpy as np
from pyvis.network import Network
from pathlib import Path

# ─── Load both graphs & distance maps ────────────────────────────────────────
base = Path(".")

# 1) Admin graph
with open(base/"admin_dag.pkl","rb") as f:
    G_admin = pickle.load(f)
with open(base/"admin_wdistances.pkl","rb") as f:
    dist_admin = pickle.load(f)

# 2) Geo proximity graph
with open(base/"geo_graph.pkl","rb") as f:
    G_geo = pickle.load(f)
with open(base/"geo_wdistances.pkl","rb") as f:
    dist_geo = pickle.load(f)

print(f"Admin Graph: {G_admin.number_of_nodes()} nodes, {G_admin.number_of_edges()} edges")
print(f"Geo Graph:   {G_geo.number_of_nodes()} nodes, {G_geo.number_of_edges()} edges")

# ─── Examine geo‐edge distance distribution ──────────────────────────────────
geo_dists = [data["weight"] for _,_,data in G_geo.edges(data=True)]
for p in [10, 25, 50, 75, 90]:
    print(f"{p}th percentile geo‐edge distance: {np.percentile(geo_dists, p):.1f} km")

# ─── Helper to visualize a graph ─────────────────────────────────────────────
def show_graph(G, out_html, notebook=True, directed=None):
    # if directed not specified, infer from G
    if directed is None:
        directed = G.is_directed()
    net = Network(notebook=notebook,
                  height="700px", width="100%",
                  directed=directed)
    color_map = {
        "country":      "red",
        "canton":       "orange",
        "district":     "lightgreen",
        "municipality":"lightblue"
    }
    for n, attrs in G.nodes(data=True):
        net.add_node(n,
                     label=attrs.get("label", n),
                     title=attrs.get("level", ""),
                     color=color_map.get(attrs.get("level",""), "gray"))
    for u, v, data in G.edges(data=True):
        label = data.get("edge_type", "admin")
        net.add_edge(u, v, title=f"{label}|{data['weight']:.1f}")
    net.show(out_html)

# ─── Visualize full admin graph (directed) ─────────────────────────────────
show_graph(G_admin,      "admin_full_graph.html", directed=True)

# ─── Prune geo graph by a cutoff (e.g. 30 km) ────────────────────────────────
MAX_GEO_KM = 30.0
geo_pruned = nx.Graph()   # undirected
geo_pruned.add_nodes_from(G_geo.nodes(data=True))
for u, v, data in G_geo.edges(data=True):
    if data["weight"] <= MAX_GEO_KM:
        geo_pruned.add_edge(u, v, **data)

print(f"Pruned Geo Graph: {geo_pruned.number_of_nodes()} nodes, "
      f"{geo_pruned.number_of_edges()} edges (≤ {MAX_GEO_KM} km)")

# ─── Visualize pruned geo graph (undirected) ───────────────────────────────
show_graph(geo_pruned,   "geo_pruned_graph.html", directed=False)


Admin Graph: 2289 nodes, 4550 edges
Geo Graph:   2128 nodes, 12339 edges
10th percentile geo‐edge distance: 2.5 km
25th percentile geo‐edge distance: 3.4 km
50th percentile geo‐edge distance: 4.7 km
75th percentile geo‐edge distance: 6.6 km
90th percentile geo‐edge distance: 9.6 km
admin_full_graph.html
Pruned Geo Graph: 2128 nodes, 12307 edges (≤ 30.0 km)
